<a href="https://colab.research.google.com/github/brs1977/BERT-Transformer-for-Summarization/blob/master/BertAbsSum2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

##Mount Google Drive

In [0]:
from google.colab import drive, files
drive.mount('/content/drive')

##Get Kaggle data

In [2]:
#!pip install -q kaggle

#kaggle key
!mkdir ~/.kaggle
!cp /content/drive/My\ Drive/kaggle.json ~/.kaggle
!ls ~/.kaggle

kaggle.json


In [3]:
!kaggle competitions download -c title-generation

 94% 41.0M/43.7M [00:00<00:00, 51.5MB/s]
100% 43.7M/43.7M [00:00<00:00, 111MB/s] 
  0% 0.00/778k [00:00<?, ?B/s]
100% 778k/778k [00:00<00:00, 108MB/s]
  0% 0.00/905k [00:00<?, ?B/s]
100% 905k/905k [00:00<00:00, 128MB/s]
  0% 0.00/6.39M [00:00<?, ?B/s]
100% 6.39M/6.39M [00:00<00:00, 58.9MB/s]


In [4]:
!mkdir data
!unzip sample_submission.csv.zip -d data
!unzip vocs.pkl.zip -d data
!unzip train.csv.zip -d data
!mv test.csv data

!ls data

Archive:  sample_submission.csv.zip
  inflating: data/sample_submission.csv  
Archive:  vocs.pkl.zip
  inflating: data/vocs.pkl           
Archive:  train.csv.zip
  inflating: data/train.csv          
sample_submission.csv  test.csv  train.csv  vocs.pkl


In [0]:
import pandas as pd

train = pd.read_csv('/content/data/train.csv', encoding='utf8')
test = pd.read_csv('/content/data/test.csv', encoding='utf8')


In [0]:
train.head()

,abstract,title
0,we consider the problem of utility maximizatio...,on optimal investment with processes of long o...
1,in this paper we provide an explicit formula f...,boolean complexes for ferrers graphs
2,"kinesin-5, also known as eg5 in vertebrates is...",relative velocity of sliding of microtubules b...
3,we discuss the transition paths in a coupled b...,bifurcation of transition paths induced by cou...
4,two types of room temperature detectors of ter...,all-electric detectors of the polarization sta...


##Requirements

In [6]:
%cd /content
!rm bertsum -r
!git clone https://github.com/brs1977/BERT-Transformer-for-Summarization bertsum

# !git pull origin master

/content
rm: cannot remove 'bertsum': No such file or directory
Cloning into 'bertsum'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 122 (delta 55), reused 80 (delta 28), pack-reused 0
Receiving objects: 100% (122/122), 135.88 KiB | 3.77 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [7]:
%cd /content/bertsum

import random
import re
import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
import argparse
import logging
import os
import json
import time
import torch.nn.functional as F
from preprocess import CSVProcessor, create_dataset
from model import BertAbsSum
from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertModel
from pytorch_pretrained_bert.optimization import BertAdam
from preprocess import convert_examples_to_features
from tqdm import tqdm, trange
from transformer import Constants

/content/bertsum


#Train

###Bert config

In [15]:
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

class ARGS(object):
    data_dir = 'data/processed_data'
    bert_model = 'bert-base-uncased'
    #output_dir = 'output'
    output_dir = '/content/drive/My Drive/nlp'
    config_path = 'config.json'
    model_path =  '/content/drive/My Drive/nlp/model_01-31-15:26:20/BertAbsSum_2.bin' #Rouge-1: 0.7403979126348502 Rouge-2: 0.593938859194881
    learning_rate = 2e-5
    change_lr = True
    load_prepared_data = True
    num_train_epochs = 5
    warmup_proportion = 0.1
    max_src_len = 130
    max_tgt_len = 30
    train_batch_size = 48
    valid_batch_size = 48
    decoder_config = None
    print_every = 100
    gradient_accumulation_steps = 8


args = ARGS()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f'Using device:{device}')

if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)
model_path = os.path.join(args.output_dir, time.strftime('model_%m-%d-%H:%M:%S', time.localtime()))
os.mkdir(model_path)
logger.info(f'Saving model to {model_path}.')



02/01/2020 08:28:05 - INFO - __main__ -   Using device:cuda
02/01/2020 08:28:05 - INFO - __main__ -   Saving model to /content/drive/My Drive/nlp/model_02-01-08:28:05.


###Data Loaders

In [9]:
if args.load_prepared_data:
  #prepared data
  train_data  = torch.load('/content/drive/My Drive/nlp/nlp_model/abs_bert/data130X30/train.pt')
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.train_batch_size, drop_last=True)

  valid_data = torch.load('/content/drive/My Drive/nlp/nlp_model/abs_bert/data130X30/valid.pt')
  valid_sampler = RandomSampler(valid_data)
  valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=args.valid_batch_size, drop_last=True)

  tokenizer = BertTokenizer.from_pretrained(args.bert_model)
else:
  is_test = False
  nrows = None if not is_test else 500

  # data preprocess
  processor = CSVProcessor()
  tokenizer = BertTokenizer.from_pretrained(args.bert_model)
  # tokenizer = BertTokenizer.from_pretrained(os.path.join(args.bert_model, 'vocab.txt'))
  logger.info('Loading train examples...')
  train_examples = processor.get_train_examples('../data/train.csv', nrows = nrows)
  num_train_optimization_steps = int(len(train_examples) / args.train_batch_size / args.gradient_accumulation_steps) * args.num_train_epochs
  logger.info('Converting train examples to features...')
  train_features = convert_examples_to_features(train_examples, args.max_src_len, args.max_tgt_len, tokenizer)
  example = train_examples[0]
  example_feature = train_features[0]
  logger.info("*** Example ***")
  logger.info("guid: %s" % (example.guid))
  logger.info("src text: %s" % example.src)
  logger.info("src_ids: %s" % " ".join([str(x) for x in example_feature.src_ids]))
  logger.info("src_mask: %s" % " ".join([str(x) for x in example_feature.src_mask]))
  logger.info("tgt text: %s" % example.tgt)
  logger.info("tgt_ids: %s" % " ".join([str(x) for x in example_feature.tgt_ids]))
  logger.info("tgt_mask: %s" % " ".join([str(x) for x in example_feature.tgt_mask]))
  logger.info('Building dataloader...')
  train_data = create_dataset(train_features)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.train_batch_size, drop_last=True)

  valid_examples = processor.get_valid_examples('../data/train.csv')
  logger.info('Converting valid examples to features...')
  valid_features = convert_examples_to_features(valid_examples, args.max_src_len, args.max_tgt_len, tokenizer)
  valid_data = create_dataset(valid_features)
  valid_sampler = RandomSampler(valid_data)
  valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=args.train_batch_size, drop_last=True)  

# save prepared data
# torch.save(train_data,'train.pt')
# torch.save(valid_data,'valid.pt')
# !cp *.pt /content/drive/My\ Drive/nlp/nlp_model/abs_bert/data130X30
# !ls /content/drive/My\ Drive/nlp/nlp_model/abs_bert/data130X30 -la


02/01/2020 08:23:27 - INFO - pytorch_pretrained_bert.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache, downloading to /tmp/tmptqugw4wn
100%|██████████| 231508/231508 [00:00<00:00, 2635955.76B/s]
02/01/2020 08:23:27 - INFO - pytorch_pretrained_bert.file_utils -   copying /tmp/tmptqugw4wn to cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
02/01/2020 08:23:27 - INFO - pytorch_pretrained_bert.file_utils -   creating metadata file for /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
02/01/2020 08:23:27 - INFO - pytorch_pretrained_bert.file_utils -   removing temp file /tmp/tmptqugw4wn
02/01/2020 08:23:27 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws

###Loss functions

In [0]:
def cal_performance(logits, ground, smoothing=True):
    ground = ground[:, 1:]
    logits = logits.view(-1, logits.size(-1))
    ground = ground.contiguous().view(-1)

    loss = cal_loss(logits, ground, smoothing=smoothing)

    pad_mask = ground.ne(Constants.PAD)
    pred = logits.max(-1)[1]
    correct = pred.eq(ground)
    correct = correct.masked_select(pad_mask).sum().item()
    return loss, correct

def cal_loss(logits, ground, smoothing=True):
    def label_smoothing(logits, labels):
        eps = 0.1
        num_classes = logits.size(-1)

        # >>> z = torch.zeros(2, 4).scatter_(1, torch.tensor([[2], [3]]), 1.23)
        # >>> z
        # tensor([[ 0.0000,  0.0000,  1.2300,  0.0000],
        #        [ 0.0000,  0.0000,  0.0000,  1.2300]])
        one_hot = torch.zeros_like(logits).scatter(1, labels.view(-1, 1), 1)
        one_hot = one_hot * (1 - eps) + (1 - one_hot) * eps / (num_classes - 1)
        log_prb = F.log_softmax(logits, dim=1)
        non_pad_mask = ground.ne(Constants.PAD)
        loss = -(one_hot * log_prb).sum(dim=1)
        loss = loss.masked_select(non_pad_mask).mean()
        return loss
    if smoothing:
        loss = label_smoothing(logits, ground)
    else:
        loss = F.cross_entropy(logits, ground, ignore_index=Constants.PAD)
    
    return loss    

def rouge(hyp, ref, n):
    scores = []
    for h, r in zip(hyp, ref):
        r = re.sub(r'[UNK]', '', r)
        r = re.sub(r'[’!"#$%&\'()*+,-./:：？！《》;<=>?@[\\]^_`{|}~]+', '', r)
        r = re.sub(r'\d', '', r)
        r = re.sub(r'[a-zA-Z]', '', r)
        count = 0
        match = 0
        for i in range(len(r) - n):
            gram = r[i:i + n]
            if gram in h:
                match += 1
            count += 1
        scores.append(0 if count==0 else match / count)
    return np.average(scores)

def convert_one_example(text, src_max_seq_length, tokenizer):
    src_tokens = tokenizer.tokenize(text)
    if len(src_tokens) > src_max_seq_length - 2:
        src_tokens = src_tokens[:(src_max_seq_length - 2)]
    src_tokens = ["[CLS]"] + src_tokens + ["[SEP]"]

    src_ids = tokenizer.convert_tokens_to_ids(src_tokens)

    src_mask = [1] * len(src_ids)
    src_padding = [0] * (src_max_seq_length - len(src_ids))
    src_ids += src_padding
    src_mask += src_padding

    return torch.tensor([src_ids]), torch.tensor([src_mask])   

def compute_lr(initAlpha,epoch,power=1,maxEpochs=5):
    # maxEpochs  : общее количество эпох, для которых мы будем тренироваться.
    # initAlpha  : начальная скорость обучения.
    # power: степень / показатель многочлена.
    # power = 1,0, линейный спад скорости обучения.
		# compute the new learning rate based on polynomial decay
		decay = (1 - (epoch / float(maxEpochs))) ** power
		alpha = initAlpha * decay
 
		# return the new learning rate
		return float(alpha)


###Validation function

In [0]:

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

def do_validate(beam_size=3, n_best=3, seed=666):
  logger.info("***** Running validation *****")
  set_seed(seed)
  model.eval()
  hyp_list = []
  ref_list = []
  i = 0
  for batch in tqdm(valid_dataloader, desc="Val iter", position=0):
      i += 1
      batch = tuple(t.to(device) for t in batch)
      pred, _ = model.beam_decode(batch[0], batch[1], beam_size=beam_size, n_best=n_best)
      src, tgt = batch[0], batch[2]
      for i in range(args.train_batch_size):
          sample_src = "".join(tokenizer.convert_ids_to_tokens(src[i].cpu().numpy())).split('[CLS]')[1].split('[SEP]')[0] + '\n'
          sample_tgt = "".join(tokenizer.convert_ids_to_tokens(tgt[i].cpu().numpy())).split('[CLS]')[1].split('[SEP]')[0] + '\n'
          sample_pred = "".join(tokenizer.convert_ids_to_tokens(pred[i][0])).split('[SEP]')[0] + '\n'

          hyp_list.append(sample_pred)
          ref_list.append(sample_tgt)
  rouge_1 = rouge(hyp_list, ref_list, 1)
  rouge_2 = rouge(hyp_list, ref_list, 2)
  logger.info('******Validation results******')
  logger.info(f'beam_size: {beam_size} n_best: {n_best} Rouge-1: {rouge_1} Rouge-2: {rouge_2}')
  logger.info('Validation finished.')

In [0]:
# do_validate()

01/28/2020 05:03:31 - INFO - __main__ -   ***** Running validation *****
Val iter: 100%|██████████| 422/422 [27:13<00:00,  3.73s/it]
01/28/2020 05:30:46 - INFO - __main__ -   ******Validation results******
01/28/2020 05:30:46 - INFO - __main__ -   Rouge-1: 0.7187769614570536
01/28/2020 05:30:46 - INFO - __main__ -   Rouge-2: 0.5851275569921026
01/28/2020 05:30:46 - INFO - __main__ -   Validation finished.


###Model

In [16]:
num_train_optimization_steps = int(len(train_data) / args.train_batch_size / args.gradient_accumulation_steps) * args.num_train_epochs

if args.model_path is None:
  model = BertAbsSum(args.bert_model, decoder_config, device=device )
  model.to(device)
else: #load checkpoint
  # args.config_path = '/content/drive/My Drive/nlp/config.json'
  with open(args.config_path, 'r') as f:
      config = json.load(f)
  logger.info(f'******Loading state model {args.model_path}******')
  model = BertAbsSum(args.bert_model, config['decoder_config'], device)
  model.load_state_dict(torch.load(args.model_path))
  model.to(device)


# optimizer
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
optimizer = BertAdam(optimizer_grouped_parameters,
                    lr=args.learning_rate,
                    warmup=0.1,
                    t_total=num_train_optimization_steps)


02/01/2020 08:28:14 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
02/01/2020 08:28:14 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpbq6jj182
02/01/2020 08:28:18 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size

###Training

In [0]:
seed = 666
# training
logger.info("***** Running training *****")
logger.info("  Num examples = %d", len(train_data))
logger.info("  Batch size = %d", args.train_batch_size)
logger.info("  Num steps = %d", num_train_optimization_steps)
model.train()
global_step = 0
for i in range(int(args.num_train_epochs)):
    set_seed(seed)
    # do training
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(tqdm(train_dataloader, desc="Train iter", position=0)):
        batch = tuple(t.to(device) for t in batch)
        logits = model(*batch)
        loss, _ = cal_performance(logits, batch[2])

        if args.gradient_accumulation_steps > 1:
            loss = loss / args.gradient_accumulation_steps
        loss.backward()
        tr_loss += loss.item()
        nb_tr_examples += batch[0].size(0)
        nb_tr_steps += 1
        if (step + 1) % args.gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
        if (step + 1) % args.print_every == 0:
            logger.info(f'Epoch {i}, step {step}, loss {loss.item()}. lr {optimizer.param_groups[0]["lr"]}.')
            logger.info(f'Ground: {"".join(tokenizer.convert_ids_to_tokens(batch[2][0].cpu().numpy()))}')
            logger.info(f'Generated: {"".join(tokenizer.convert_ids_to_tokens(logits[0].max(-1)[1].cpu().numpy()))}')
    
    #do save model
    if args.output_dir is not None:
        state_dict = model.state_dict()
        torch.save(state_dict, os.path.join(model_path, 'BertAbsSum_{}.bin'.format(i)))
        logger.info('Model saved')
    
    # do evaluation
    if valid_dataloader is not None:
        model.eval()
        batch = next(iter(valid_dataloader))
        batch = tuple(t.to(device) for t in batch)
        # beam_decode
        pred, _ = model.beam_decode(batch[0], batch[1], 3, 3)
        # pred = model.greedy_decode(batch[0], batch[1])
        logger.info(f'Source: {"".join(tokenizer.convert_ids_to_tokens(batch[0][0].cpu().numpy()))}')
        logger.info(f'Beam Generated: {"".join(tokenizer.convert_ids_to_tokens(pred[0][0]))}')
        # logger.info(f'Beam Generated: {tokenizer.convert_ids_to_tokens(pred[0].cpu().numpy())}')
    
    # do validate        
    do_validate()

    if args.change_lr:
      optimizer.param_groups[0]['lr'] = compute_lr(args.learning_rate,i)
      optimizer.param_groups[1]['lr'] = compute_lr(args.learning_rate,i)
      logger.info(f'Epoch {i} change lr {optimizer.param_groups[0]["lr"]}.')

    logger.info(f'Epoch {i} finished.')
with open(os.path.join(args.bert_model, 'bert_config.json'), 'r') as f:
    bert_config = json.load(f)
config = {'bert_config': bert_config, 'decoder_config': decoder_config}
with open(os.path.join(model_path, 'config.json'), 'w') as f:
    json.dump(config, f)
logger.info('Training finished')

02/01/2020 08:30:30 - INFO - __main__ -   ***** Running training *****
02/01/2020 08:30:30 - INFO - __main__ -     Num examples = 121489
02/01/2020 08:30:30 - INFO - __main__ -     Batch size = 48
02/01/2020 08:30:30 - INFO - __main__ -     Num steps = 1580
Train iter:   4%|▍         | 99/2531 [02:25<1:01:29,  1.52s/it]02/01/2020 08:32:57 - INFO - __main__ -   Epoch 0, step 99, loss 0.31639933586120605. lr 2e-05.
02/01/2020 08:32:57 - INFO - __main__ -   Ground: [CLS]theho##lom##or##phyconjectureforidealsindimensiontwo[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]
02/01/2020 08:32:57 - INFO - __main__ -   Generated: theho##lom##or##phyconjectureforsubinan-[SEP][SEP]----------------
Train iter:   8%|▊         | 199/2531 [05:01<1:01:15,  1.58s/it]02/01/2020 08:35:33 - INFO - __main__ -   Epoch 0, step 199, loss 0.28668951988220215. lr 2e-05.
02/01/2020 08:35:33 - INFO - __main__ -   Ground: [CLS]lowenergypropertiesofthesu(m|n)super##sy##mme##tr

In [1]:
!ls


sample_data


#Predict

###Load model

In [0]:
class ARGS(object):
    # data_dir = 'data/processed_data'
    bert_model = 'bert-base-uncased'
    output_dir = 'output'
    model_path =  'output/model_01-13-05:43:53/BertAbsSum_1.bin'
    config_path = 'output/model_01-13-05:43:53/config.json'
    result_path = 'result'
    batch_size = 16
    max_src_len = 512

args = ARGS()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
with open(args.config_path, 'r') as f:
    config = json.load(f)

model = BertAbsSum(args.bert_model, config['decoder_config'], device)
model.load_state_dict(torch.load(args.model_path))
model.to(device)

###Generate titles

In [54]:
set_seed(1616161)

import pandas as pd
from tqdm import tqdm
# test = pd.read_csv('../data/test.csv', encoding='utf8')

titles = []

for row in tqdm(test.iterrows(), desc="Iteration", total = test.shape[0], position=0):
  i, text = row
  src, src_mask = convert_one_example(text[0], args.max_src_len, tokenizer)
  pred, _ = model.beam_decode(src.to(device), src_mask.to(device), 5, 5)  
  # print(" ".join(tokenizer.convert_ids_to_tokens(pred[0][0])).split('[SEP]')[0])

  # De-tokenize.
  tok_text = " ".join(tokenizer.convert_ids_to_tokens(pred[0][0])).split('[SEP]')[0]
  tok_text = tok_text.replace(" ##", "")
  tok_text = tok_text.replace("##", "")  
  
  # tok_text = ''
  # for t in tokenizer.convert_ids_to_tokens(pred[0][0]):
  #   tok_text += ' ' + t if not t.startswith('##') else t[2:]
  # tok_text = tok_text.split('[SEP]')[0][1:]
  titles.append(tok_text)

submission_df = pd.DataFrame({'abstract': test.abstract, 'title': titles})
submission_df.to_csv('predicted_titles.csv', index=False)  

Iteration: 100%|██████████| 1000/1000 [04:10<00:00,  3.75it/s]


###Submission

In [56]:
import string
from nltk.util import ngrams
import numpy as np
import pandas as pd
import pickle


def generate_csv(input_file='predicted_titles.csv',
                 output_file='submission.csv',
                 voc_file='../data/vocs.pkl'):
    '''
    Generates file in format required for submitting result to Kaggle
    
    Parameters:
        input_file (str) : path to csv file with your predicted titles.
                           Should have two fields: abstract and title
        output_file (str) : path to output submission file
        voc_file (str) : path to voc.pkl file
    '''
    data = pd.read_csv(input_file)
    with open(voc_file, 'rb') as voc_file:
        vocs = pickle.load(voc_file)

    with open(output_file, 'w') as res_file:
        res_file.write('Id,Predict\n')
        
    output_idx = 0
    for row_idx, row in data.iterrows():
        trg = row['title']
        trg = trg.translate(str.maketrans('', '', string.punctuation)).lower().split()
        trg.extend(['_'.join(ngram) for ngram in list(ngrams(trg, 2)) + list(ngrams(trg, 3))])
        
        VOCAB_stoi = vocs[row_idx]
        trg_intersection = set(VOCAB_stoi.keys()).intersection(set(trg))
        trg_vec = np.zeros(len(VOCAB_stoi))    

        for word in trg_intersection:
            trg_vec[VOCAB_stoi[word]] = 1

        with open(output_file, 'a') as res_file:
            for is_word in trg_vec:
                res_file.write('{0},{1}\n'.format(output_idx, int(is_word)))
                output_idx += 1


generate_csv()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: generator 'ngrams' raised StopIteration


In [58]:
!kaggle competitions submit -c title-generation -m "model_01-30-05:17:03/BertAbsSum_1.bin 0.7321 Rouge-2: 0.5893" -f submission.csv 



100% 2.84M/2.84M [00:02<00:00, 1.04MB/s]
Successfully submitted to Arxiv Title Generation

In [0]:
!ls -la